# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 23 2022 1:52PM,247169,19,N17780,"NAPP Technologies, LLC",Released
1,Sep 23 2022 1:42PM,247168,20,8475565,"ACI Healthcare USA, Inc.",Executing
2,Sep 23 2022 1:42PM,247168,20,8546422,"ACI Healthcare USA, Inc.",Executing
3,Sep 23 2022 1:42PM,247168,20,8546435-BO,"ACI Healthcare USA, Inc.",Executing
4,Sep 23 2022 1:42PM,247168,20,8559674,"ACI Healthcare USA, Inc.",Released
5,Sep 23 2022 1:39PM,247167,10,8564545,"Citieffe, Inc.",Executing
6,Sep 23 2022 1:03PM,247165,10,CTF0011426.1,"Citieffe, Inc.",Released
7,Sep 23 2022 1:03PM,247165,10,CTF0011427,"Citieffe, Inc.",Released
8,Sep 23 2022 12:40PM,247162,10,0086005871,ISDIN Corporation,Released
9,Sep 23 2022 12:40PM,247162,10,0086005872,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
23,247165,Released,2
24,247167,Executing,1
25,247168,Executing,3
26,247168,Released,1
27,247169,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247162,NaN,NaN,7.0
247165,NaN,NaN,2.0
247167,NaN,1.0,NaN
247168,NaN,3.0,1.0
247169,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246891,0.0,1.0,0.0
247083,5.0,1.0,4.0
247107,0.0,0.0,1.0
247115,0.0,1.0,0.0
247116,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246891,0,1,0
247083,5,1,4
247107,0,0,1
247115,0,1,0
247116,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246891,0,1,0
1,247083,5,1,4
2,247107,0,0,1
3,247115,0,1,0
4,247116,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246891,,1,
1,247083,5,1,4
2,247107,,,1
3,247115,,1,
4,247116,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 23 2022 1:52PM,247169,19,"NAPP Technologies, LLC"
1,Sep 23 2022 1:42PM,247168,20,"ACI Healthcare USA, Inc."
5,Sep 23 2022 1:39PM,247167,10,"Citieffe, Inc."
6,Sep 23 2022 1:03PM,247165,10,"Citieffe, Inc."
8,Sep 23 2022 12:40PM,247162,10,ISDIN Corporation
15,Sep 23 2022 12:36PM,247161,16,Sartorius Bioprocess Solutions
19,Sep 23 2022 11:49AM,247151,10,ISDIN Corporation
42,Sep 23 2022 11:44AM,247150,10,ISDIN Corporation
55,Sep 23 2022 10:58AM,247117,15,"Alliance Pharma, Inc."
72,Sep 23 2022 10:47AM,247146,16,TASA USA Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 23 2022 1:52PM,247169,19,"NAPP Technologies, LLC",,,1
1,Sep 23 2022 1:42PM,247168,20,"ACI Healthcare USA, Inc.",,3,1
2,Sep 23 2022 1:39PM,247167,10,"Citieffe, Inc.",,1,
3,Sep 23 2022 1:03PM,247165,10,"Citieffe, Inc.",,,2
4,Sep 23 2022 12:40PM,247162,10,ISDIN Corporation,,,7
5,Sep 23 2022 12:36PM,247161,16,Sartorius Bioprocess Solutions,,,4
6,Sep 23 2022 11:49AM,247151,10,ISDIN Corporation,,,23
7,Sep 23 2022 11:44AM,247150,10,ISDIN Corporation,,4,9
8,Sep 23 2022 10:58AM,247117,15,"Alliance Pharma, Inc.",,13,4
9,Sep 23 2022 10:47AM,247146,16,TASA USA Inc.,,1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 23 2022 1:52PM,247169,19,"NAPP Technologies, LLC",1,,
1,Sep 23 2022 1:42PM,247168,20,"ACI Healthcare USA, Inc.",1,3,
2,Sep 23 2022 1:39PM,247167,10,"Citieffe, Inc.",,1,
3,Sep 23 2022 1:03PM,247165,10,"Citieffe, Inc.",2,,
4,Sep 23 2022 12:40PM,247162,10,ISDIN Corporation,7,,
5,Sep 23 2022 12:36PM,247161,16,Sartorius Bioprocess Solutions,4,,
6,Sep 23 2022 11:49AM,247151,10,ISDIN Corporation,23,,
7,Sep 23 2022 11:44AM,247150,10,ISDIN Corporation,9,4,
8,Sep 23 2022 10:58AM,247117,15,"Alliance Pharma, Inc.",4,13,
9,Sep 23 2022 10:47AM,247146,16,TASA USA Inc.,,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 23 2022 1:52PM,247169,19,"NAPP Technologies, LLC",1,,
1,Sep 23 2022 1:42PM,247168,20,"ACI Healthcare USA, Inc.",1,3,
2,Sep 23 2022 1:39PM,247167,10,"Citieffe, Inc.",,1,
3,Sep 23 2022 1:03PM,247165,10,"Citieffe, Inc.",2,,
4,Sep 23 2022 12:40PM,247162,10,ISDIN Corporation,7,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 23 2022 1:52PM,247169,19,"NAPP Technologies, LLC",1.0,NaN,NaN
1,Sep 23 2022 1:42PM,247168,20,"ACI Healthcare USA, Inc.",1.0,3.0,NaN
2,Sep 23 2022 1:39PM,247167,10,"Citieffe, Inc.",NaN,1.0,NaN
3,Sep 23 2022 1:03PM,247165,10,"Citieffe, Inc.",2.0,NaN,NaN
4,Sep 23 2022 12:40PM,247162,10,ISDIN Corporation,7.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 23 2022 1:52PM,247169,19,"NAPP Technologies, LLC",1.0,0.0,0.0
1,Sep 23 2022 1:42PM,247168,20,"ACI Healthcare USA, Inc.",1.0,3.0,0.0
2,Sep 23 2022 1:39PM,247167,10,"Citieffe, Inc.",0.0,1.0,0.0
3,Sep 23 2022 1:03PM,247165,10,"Citieffe, Inc.",2.0,0.0,0.0
4,Sep 23 2022 12:40PM,247162,10,ISDIN Corporation,7.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2718565,56.0,16.0,6.0
15,247117,4.0,13.0,0.0
16,741414,5.0,1.0,0.0
19,247169,1.0,0.0,0.0
20,247168,1.0,3.0,0.0
21,741122,1.0,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2718565,56.0,16.0,6.0
1,15,247117,4.0,13.0,0.0
2,16,741414,5.0,1.0,0.0
3,19,247169,1.0,0.0,0.0
4,20,247168,1.0,3.0,0.0
5,21,741122,1.0,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,56.0,16.0,6.0
1,15,4.0,13.0,0.0
2,16,5.0,1.0,0.0
3,19,1.0,0.0,0.0
4,20,1.0,3.0,0.0
5,21,1.0,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,56.0
1,15,Released,4.0
2,16,Released,5.0
3,19,Released,1.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21
Status,,,,,,
Completed,6.0,0.0,0.0,0.0,0.0,0.0
Executing,16.0,13.0,1.0,0.0,3.0,2.0
Released,56.0,4.0,5.0,1.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21
0,Completed,6.0,0.0,0.0,0.0,0.0,0.0
1,Executing,16.0,13.0,1.0,0.0,3.0,2.0
2,Released,56.0,4.0,5.0,1.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21
0,Completed,6.0,0.0,0.0,0.0,0.0,0.0
1,Executing,16.0,13.0,1.0,0.0,3.0,2.0
2,Released,56.0,4.0,5.0,1.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()